# Querying and Creating Portfolios in VAM Client
This notebook serves as a tutorial for querying, creating, and managing portfolios using the `mainsequence.client` package. It covers:
- Querying existing portfolios using filters
- Creating portfolios from time series
- Creating an index asset portfolio
- Querying the newly created index asset portfolio

## Step 1: Import Required Modules
First, we import the necessary modules and constants.

In [1]:
import os
from pathlib import Path
import dotenv
# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

import dotenv
dotenv.load_dotenv('.env')
from mainsequence.client import DataUpdates
import datetime
from mainsequence.client import TargetPortfolio, Asset,TargetPortfolioIndexAsset
from mainsequence.client import MARKETS_CONSTANTS


Working directory set to: /Users/jose/code/mainsequence-sdk


2025-05-19T13:43:39.289443Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:39 in refresh_headers())
2025-05-19T13:43:39.531581Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:39 in refresh_headers())
2025-05-19T13:43:39.962387Z [debug    ] took 0.4010 seconds. Requesting GET from http://192.168.178.69:8000/orm/api/pods/projects/get_user_default_project/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-19T13:43:39.963482Z [info     ] Set remote data source to orm_class='DataSource' id=1 display_name='Default TS GCP' organization=1 class_type='timescale_db' status='AVAILABLE' extra_arguments=None application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at models_tdag.py:2114 in set_remote_db())


## Step 2: Query Portfolios
We query an existing portfolio using the `local_time_serie_hash_id` filter. This returns a single portfolio object.

In [2]:

# Query a single TargetPortfolio with a specific ticker
portfolio_ticker = "TICKER1"
portfolio = TargetPortfolio.filter(portfolio_ticker=portfolio_ticker)
print("Queried Portfolio:", portfolio)


2025-05-19T13:43:40.474028Z [debug    ] took 0.4497 seconds. Requesting GET from http://192.168.178.69:8000/orm/api/assets/target_portfolio/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Queried Portfolio: []


## Step 3: Create Portfolios from Time Series
We create two portfolios for different purposes:
- **Execution Portfolio** for real-time execution
- **Backtesting Portfolio** for simulation and historical analysis

### !!! Important !!!

If you want to start building portfolios we recommend you to use our VirtualFundBuilder. This will help you have a more fluid 
and seamless workflows. In this example we will show you how to build a portfolio without using our Wrapper. A few important considerations

1.  Each portfolio requires a `local_signal_time_serie` and a `local_time_serie` that should come from TDAG, this is a unique identifier in the backend and placing a random integer will likely collide therefore we need to build a time serie that has a time_index and at least a column with a column "close" 



In [3]:
from mainsequence.tdag.time_series import TimeSerie
from mainsequence.client import DataUpdates
import numpy as np
import datetime
import pandas as pd

class PortfolioExample(TimeSerie):
    
    @TimeSerie._post_init_routines()
    def __init__(self, portfolio_volatility: float,*args, **kwargs):
        self.portfolio_volatility = portfolio_volatility
        super().__init__(*args, **kwargs)
        
    def update(self, update_statistics):
        """
        Update the time series by simulating a new data point based on the last observation,
        or simulate a complete series if no observations exist.

        :param update_statistics: A dict to record update statistics (optional).
        :return: pd.DataFrame with index as a timezone-aware datetime (UTC) and a column "close".
        """
        last_observation = self.get_last_observation()
        dt = 1.0  # one day time increment

        mock_asset=Asset.filter(ticker="NVDA",execution_venue=MARKETS_CONSTANTS.MAIN_SEQUENCE_EV)[0]

        if last_observation is not None:
            return pd.DataFrame() # do not make sequential updats for example
          
        else:
            # If no observation exists, simulate a daily series for the last 30 days.
            end_date = datetime.datetime.now(datetime.timezone.utc)
            start_date = end_date - datetime.timedelta(days=30)
            # Generate a date range with daily frequency (timezone-aware in UTC).
            dates = pd.date_range(start=start_date, end=end_date, freq='D', tz=datetime.timezone.utc)
            
            # Initialize with a default price.
            prices = [100.0]
            for _ in range(1, len(dates)):
                random_return = np.random.normal(loc=0, scale=self.portfolio_volatility * np.sqrt(dt))
                new_price = prices[-1] * np.exp(random_return - 0.5 * self.portfolio_volatility**2 * dt)
                prices.append(new_price)
                
            new_data = pd.DataFrame({"close": prices,"rebalance_weights":{mock_asset.unique_identifier:1.0},
                                     "rebalance_price":{mock_asset.unique_identifier:1.0},
                                     "weights_at_last_rebalance":{mock_asset.unique_identifier:1.0},
                                      "price_at_last_rebalance":{mock_asset.unique_identifier:1.0},
                                        "volume_at_last_rebalance":{mock_asset.unique_identifier:1.0},



                                     }, index=dates)
            new_data["last_rebalance_date"]=new_data.index
            new_data["close_time"]=new_data.index
            new_data["return"]=new_data["close"].pct_change()
        if last_observation is not None:
            new_data=new_data[new_data.index>update_statistics._max_time_in_update_statistics]
         
        return new_data
    
    
portfolio_ts=PortfolioExample(portfolio_volatility=.1)
portfolio_ts.run(debug_mode=True,force_update=True)


2025-05-19T13:43:40.503995Z [debug    ] local/remote portfolioexample_f53f240ed0980b8f8bd541172af036ac/portfolioexample_f53f240ed0980b8f8bd541172af036ac application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at time_series.py:2143 in _create_config())
Overriding of current TracerProvider is not allowed
2025-05-19T13:43:41.840985Z [debug    ] took 0.8457 seconds. Requesting POST from http://192.168.178.69:8000/orm/api/ogm/scheduler/build_and_assign_to_ts/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-19T13:43:41.842241Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:39 in refresh_headers())
2025-05-19T13:43:42.053878Z [debug    ] took 0.2109 seconds. Requesting GET from http://192.168.178.69:8000/orm/api/ts_manager/api/constants application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_reque

In [4]:
#we can get our simulated prices from TDAG Backend
print(f"Data in time series",portfolio_ts)
portfolio_ts.get_df_between_dates()

Data in time series PortfolioExample http://192.168.178.69:8000/local-time-series/details/?local_time_serie_id=255


2025-05-19T13:44:01.433937Z [debug    ] took 0.6898 seconds. Requesting POST from http://192.168.178.69:8000/orm/api/ts_manager/local_time_serie/255/get_data_between_dates_from_remote/ api_time_series=False application_name=ms-sdk data_source_id=1 head_local_ts_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=1 scheduler_name=DEBUG_portfolioexample_f53f240ed0980b8f8bd541172af036ac_1 (at utils.py:95 in make_request())


,close
time_index,
2025-04-19 13:43:50.382000+00:00,100.000000
2025-04-20 13:43:50.382000+00:00,97.834043
2025-04-21 13:43:50.382000+00:00,104.090606
2025-04-22 13:43:50.382000+00:00,95.882577
2025-04-23 13:43:50.382000+00:00,113.441587
2025-04-24 13:43:50.382000+00:00,124.150442
2025-04-25 13:43:50.382000+00:00,108.641368
2025-04-26 13:43:50.382000+00:00,120.532926
2025-04-27 13:43:50.382000+00:00,108.228648


In [10]:

def create_portfolio(build_purpose, portfolio_name,portfolio_ts,valuation_asset):
    
    
    existing_portfolio = TargetPortfolio.get_or_none(
       
        local_time_serie__id=portfolio_ts.local_metadata.id
    )
    
    if existing_portfolio:
        print(f"Portfolio '{portfolio_name}' already exists.")
        index_asset=TargetPortfolioIndexAsset.get(reference_portfolio__id=existing_portfolio.id)
        
        
        return existing_portfolio,index_asset
    
    
    return TargetPortfolio.create_from_time_series(
        portfolio_name=portfolio_name,
        build_purpose=build_purpose,
        is_active=True,
        local_time_serie_id=portfolio_ts.local_metadata.id,  # Example ID
        signal_local_time_serie_id=portfolio_ts.local_metadata.id, #we are not using a signal so we are setting the same 
        required_venues__symbols=[MARKETS_CONSTANTS.BINANCE_EV_SYMBOL],
        calendar_name="24/7",
        tracking_funds_expected_exposure_from_latest_holdings=False, 
        is_asset_only=False,
        backtest_table_price_column_name="close",
        valuation_asset_id=valuation_asset.id,
        target_portfolio_about=dict(description= "Test Portfolio For example",
                                    signal_name= "No signal",
                                    signal_description="no descritpiont",
                                    rebalance_strategy_name="no rebalance"), timeout=600000
    )

valuation_asset=Asset.get(ticker="USDT",execution_venue__symbol=MARKETS_CONSTANTS.BINANCE_EV_SYMBOL)
# Create Backtesting Portfolio
backtest_portfolio, portfolio_index_asset = create_portfolio(
    MARKETS_CONSTANTS.PORTFOLIO_BUILD_FOR_BACKTEST, "Example Portfolio",portfolio_ts,valuation_asset,
)
print("Created Backtest Portfolio:", backtest_portfolio.portfolio_name)
print("Related Asset", valuation_asset)


2025-05-19T13:47:09.268763Z [debug    ] took 0.5354 seconds. Requesting GET from http://192.168.178.69:8000/orm/api/assets/asset/ api_time_series=False application_name=ms-sdk data_source_id=1 head_local_ts_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=1 scheduler_name=DEBUG_portfolioexample_f53f240ed0980b8f8bd541172af036ac_1 (at utils.py:95 in make_request())
2025-05-19T13:47:09.682028Z [debug    ] took 0.4120 seconds. Requesting GET from http://192.168.178.69:8000/orm/api/assets/target_portfolio/ api_time_series=False application_name=ms-sdk data_source_id=1 head_local_ts_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=1 scheduler_name=DEBUG_portfolioexample_f53f240ed0980b8f8bd541172af036ac_1 (at utils.py:95 in make_request())
2

Created Backtest Portfolio: Example Portfolio
Related Asset orm_class='Asset' id=23020 can_trade=False calendar=Calendar: 1 execution_venue=ExecutionVenue: 2 delisted_datetime=None unique_identifier='USDT_bnce_KKG000000H64' real_figi=False figi='KKG000000H64' composite='KKG000000H64' ticker='USDT' security_type='Crypto' security_type_2='CRYPTO' security_market_sector='Curncy' share_class='KKG000000H64' exchange_code=None name='tether' main_sequence_share_class='LwZ5U4MvuuIH'
